# Micrograd

This notebook has two different types of NNs, both follow the first of the video lectures by Andrej Karpathy. The demo notebooks are also extracted from his personal (github)[https://github.com/karpathy/micrograd].

Micrograd is an implementation of backpropagation from the scratch. It defines the class Value and the required operations (addition, multiplication, etc).

The second code is a standard implementation of NN architecture using Pytorch. Micrograd is intended to be a small but powerful "homemade" version of this.


Finally, draw_dot is a function that employs graphviz to reconstruct visually a NN with all the involved operations.

In [1]:
#Micrograd implementation

import math
import numpy as np
import matplotlib.pyplot as plt
import graphviz

%matplotlib inline

from graphviz import Digraph

# draw_dot

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            #if this value is a result of some operation, create an op node for it
            dot.node(name= uid + n._op, label = n._op)
            #and connect this node to it
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        #connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot



# micrograd

class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label= label
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __radd__ (self, other): #other + self
        return self + other
    
    def __neg__(self):
        return self *-1
    
    def __sub__(self, other): #self - other
        return self + (-other)
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward    
        return out
    
    def __rmul__ (self, other): # other * self
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/floats for now"
        out = Value(self.data**other, (self,), f'**{other}')             
        def _backward():
            self.grad +=  other* (self.data ** (other-1)) * out.grad
        out._backward = _backward
        return out
    
    def __truediv__(self, other): # self /other
        return self * other**-1
    
    def tanh(self):
        x= self.data
        
        t= (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1- t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x= self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data *out.grad
        out._backward = _backward
        return out
    
        
    def backward(self):

        topo = []
        visited= set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                        build_topo(child)
                topo.append(v)     
        build_topo(self)
          #change o to self to make it universal      
        self.grad =1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
# example for micrograd

#inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
#weights and bias
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
b = Value(6.8813737, label='b')

#inputs * weights and adding the bias
x1w1 = x1 *w1; x1w1.label = 'x1*w1'
x2w2 = x2 *w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1 + x2w2'
n = x1w1x2w2 + b; n.label = 'n'

#----
e= (2*n).exp()
o = (e -1)/(e +1);o.label='o'

#----
o.backward()
draw_dot(o)

In [ ]:
# Pytorch implementation

#lets gonna create a Neuron class compatible with PyTorch that applies forward

class Neuron:
    #Neuron uses two inputs: nin is number of inputs, and matches (not necessarily)t he number of elements of input x
    def __init__(self, nin):
        self.w = [Value(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(np.random.uniform(-1,1))

    def __call__(self,x):
        #this code implements the forward pass, w * x + b
        act = sum(wi*xi for wi, xi in zip(self.w,x)) + self.b
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
#to implement a single neuron, firstly we call the number of inputs, n= Neuron(2), and then call the input, n(x)
#Neuron ignores the rest of X if numx if lenght x > nin, and assumes 0 if nin > lenght(x)

#now we implement a layer of neurons. 'nouts' is the number of outputs (neurons) in the layer. 
#layer requires two inputs: number of inputs and number of neurons

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    #the previous expression is equal to the next 5:
        #params = []
        #for neuron in self.neurons:
            #ps = neuron.parameters()
            #params.extend(ps)
        #return params

    #multilayer perceptrop receives number of inputs,  number of output layers (nouts)
class MLP:
    
    def __init__ (self, nin, nouts):
        size = [nin] + nouts
        self.layers = [Layer(size[i], size[i+1]) for i in range (len(nouts))]
    
    def __call__ (self, x):
        for layer in self.layers:
            x= layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
        
    


In [ ]:
# Pytorch implementation

#complete implementation
x= [2.0, 3.0, -1.0]
n = MLP(3, [4,4,1])
n(x)

len(n.parameters())


xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys= [1.0, -1.0, -1.0, 1.0]  #desired targets


for k in range(20):
    #forward pass
    ypred = [n(x)  for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    
    #backward pass, remember to reset grad every time!
    for p in n.parameters():
        p.grad = 0.0
    
    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.01 * p.grad
        
    print(k, loss.data)
        
ypred        